In [1]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_pickle(PATH + 'inter/train_only_0.pkl')
test = pd.read_pickle(PATH + 'inter/test_only_0.pkl')

train.shape, test.shape

((307511, 329), (48744, 328))

In [ ]:

for x in os.listdir(PATH + 'inter/'):
    if 'curr' in x:
        if x != 'bureau2curr_2.pkl' and x!= '06_27bureau2curr.pkl':
            print(x)
            tmp = pd.read_pickle(PATH + 'inter/' + x)
            print(tmp.shape)
            col = [i for i in tmp.columns]
            train = train.merge(tmp[col], on='SK_ID_CURR', how='left')
            test = test.merge(tmp[col], on='SK_ID_CURR', how='left')
            print('train&test, shape',train.shape, test.shape)
            del tmp
            gc.collect()
        

for x in os.listdir(PATH + 'inter/linear/'):
    if x.split('.')[-1] == 'pkl':
        print(x)
        tmp = pd.read_pickle(PATH + 'inter/linear/' + x)
        col = [i for i in tmp.columns]
        train = train.merge(tmp[col], on='SK_ID_CURR', how='left')
        test = test.merge(tmp[col], on='SK_ID_CURR', how='left')
        del tmp
        gc.collect()
        print('train&test, shape',train.shape, test.shape)

'done'

In [ ]:
train.to_pickle(PATH + 'train_bureau_3_with_linear.pkl')
test.to_pickle(PATH + 'test_bureau_3_with_linear.pkl')
train.shape

In [4]:
import os
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'

In [5]:
# Read train and test
train = pd.read_pickle(PATH + 'train_factorized_2_with_linear.pkl')
test = pd.read_pickle(PATH + 'test_factorized_2_with_linear.pkl')
train.shape, test.shape

((307511, 685), (48744, 684))

In [6]:
def model_2(train_df, test_df, threshold1, threshold2, n_splits):
    print('begin cv')
    target = train_df['TARGET']
    ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
    features = [x for x in train_df.columns if x not in ignore_cols]
    train = train_df[features]
    test = test_df[features]

    from sklearn.model_selection import StratifiedKFold
    from lightgbm import LGBMClassifier
    from sklearn.metrics import roc_auc_score
    import lightgbm as lgbm

    lgbm_train = lgbm.Dataset(data=train,
                              label=target,

                              categorical_feature=[],#categorical_feats,
                              free_raw_data=False)
    
    lgbm_params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_iteration': 4000,
        'num_threads': 16,
        
        'num_leaves': int(round(44.368535336628419)),
        'feature_fraction': 0.28231763168020257,
        'bagging_fraction': 0.94901525271474951,
        'max_depth': int(round(8.0430115561596267)),
        'lambda_l1': 0.30680079516647751,
        'lambda_l2': 0.079128660903201031,
        'min_split_gain': 0.054005067457890979,
        'min_child_weight': 98.172643147364937
    }

    cv_results = lgbm.cv(train_set=lgbm_train,
                         params=lgbm_params,
                         nfold=n_splits,
                         seed=2018,
                         early_stopping_rounds=150,
                         verbose_eval=100,
                         metrics=['auc'])

    optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
    print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
    print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

    clf = lgbm.train(train_set=lgbm_train,
                     params=lgbm_params,
                     num_boost_round=optimum_boost_rounds)
    return out_df, small_len, large_len, clf

In [ ]:
n_splits = 5
pred1, small, large, clf = model_2(train, test, 0.1, 0.2,n_splits)

begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.773998 + 0.00371019
[200]	cv_agg's auc: 0.784219 + 0.00345067
[300]	cv_agg's auc: 0.787647 + 0.00310528
[400]	cv_agg's auc: 0.78917 + 0.00309462
[500]	cv_agg's auc: 0.790046 + 0.00317421
[600]	cv_agg's auc: 0.790345 + 0.00308064
[700]	cv_agg's auc: 0.790336 + 0.0029856
Optimum boost rounds = 638
Best CV result = 0.7903991373954783


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


ALL
[100]	cv_agg's auc: 0.7795 + 0.00411813
[200]	cv_agg's auc: 0.789489 + 0.0034798
[300]	cv_agg's auc: 0.792526 + 0.00351702
[400]	cv_agg's auc: 0.7936 + 0.00338604
[500]	cv_agg's auc: 0.794217 + 0.00323238
[600]	cv_agg's auc: 0.794211 + 0.00315142
[700]	cv_agg's auc: 0.794229 + 0.00328479
Optimum boost rounds = 550
Best CV result = 0.7943004497519002

sth2prev2curr no sum
[100]	cv_agg's auc: 0.777611 + 0.00437966
[200]	cv_agg's auc: 0.784903 + 0.00415943
[300]	cv_agg's auc: 0.786765 + 0.00370779
[400]	cv_agg's auc: 0.787215 + 0.00346352
[500]	cv_agg's auc: 0.787577 + 0.00352841 

In [8]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_pickle(PATH + 'inter/train_only_0.pkl')
test = pd.read_pickle(PATH + 'inter/test_only_0.pkl')

train.shape, test.shape

for x in os.listdir(PATH + 'inter/'):
    if 'curr' in x:
        if x != '06_27bureau2curr.pkl':
            print(x)
            tmp = pd.read_pickle(PATH + 'inter/' + x)
            print(tmp.shape)
            col = [i for i in tmp.columns]
            train = train.merge(tmp[col], on='SK_ID_CURR', how='left')
            test = test.merge(tmp[col], on='SK_ID_CURR', how='left')
            print('train&test, shape',train.shape, test.shape)
            del tmp
            gc.collect()
        

for x in os.listdir(PATH + 'inter/linear/'):
    if x.split('.')[-1] == 'pkl':
        print(x)
        tmp = pd.read_pickle(PATH + 'inter/linear/' + x)
        col = [i for i in tmp.columns]
        train = train.merge(tmp[col], on='SK_ID_CURR', how='left')
        test = test.merge(tmp[col], on='SK_ID_CURR', how='left')
        del tmp
        gc.collect()
        print('train&test, shape',train.shape, test.shape)

'done'

n_splits = 5
pred1, small, large, clf = model_2(train, test, 0.1, 0.2,n_splits)

poscash2curr.pkl
(356255, 101)
train&test, shape (307511, 429) (48744, 428)
bureau2curr.pkl
(356255, 460)
train&test, shape (307511, 888) (48744, 887)
prev2curr.pkl
(356255, 536)
train&test, shape (307511, 1423) (48744, 1422)
credit2curr.pkl
(356255, 382)
train&test, shape (307511, 1804) (48744, 1803)
install2curr.pkl
(356255, 51)
train&test, shape (307511, 1854) (48744, 1853)
linearposcash2curr.pkl
train&test, shape (307511, 1858) (48744, 1857)
linearbureau2curr.pkl
train&test, shape (307511, 1874) (48744, 1873)
linearinstal2curr.pkl
train&test, shape (307511, 1880) (48744, 1879)
linearcredit2curr.pkl
train&test, shape (307511, 1894) (48744, 1893)
begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.7795 + 0.00411813


KeyboardInterrupt: 